## Лабораторная работа №2
### Решение СЛАУ прямым и итерацинным методом
#### Задание II.10.5(пункт у):
Решить методами Гаусса и Зейделя, найти λmin, λmax, определить число обусловленности матрицы μ = ||A||·||A^{-1}||. Сделать печать невязок обоих методов.

Результаты:

--------------------------------------------------------------------------------------

min|λ(A)|, max|λ(A)| =  0.7357016342479968 1.519941391528549

μ = ||A||*||A^(-1)|| =  1.617256887091802

--------------------------------------------------------------------------------------

Roots of LU method:
[0.75537042 0.23879963 0.17890325 0.15086988 0.13170938 0.1170332
 0.10527117 0.09560544 0.08752166 0.08066593 0.074783   0.06968322]
 
Solution Discrepancy |A * x - f|: 6.655553224256149e-17

--------------------------------------------------------------------------------------

Roots of relaxation method
[0.75537042 0.23879963 0.17890325 0.15086988 0.13170938 0.1170332
 0.10527117 0.09560544 0.08752166 0.08066593 0.074783   0.06968322]

Solution Discrepancy |A * x - f|: 3.626858354025536e-16

Number of iterations: 41

In [141]:
import numpy as np
import random

In [143]:
# Creating matrix
n = 12                     # size of matrix nxn
matrix = np.zeros ((n, n)) # initialization 

# Filling matrix
for i in range(n):
    for j in range(n):
        if i == j:
            matrix[i][j] = 1
        else:
            matrix[i][j] = 1 / ((i + 1)*(i + 1) + (j+1))

f = np.array ([1 / (i + 1) for i in range(n)]) # right side of the system of equations
x = np.zeros (n) # init solution

In [142]:
# Printing matrix
def printMatrix(M):
    print(f"Matrix[{M.shape[0]}][{M.shape[1]}]")
    rows = M.shape[0]
    cols = M.shape[1]
    for i in range(0, rows):
        for j in range(0, cols):
            start_symbol = ""
            end_symbol   = ""

            if j == 0:
                if i == 0:
                    start_symbol = "/ "
                elif i == rows - 1:
                    start_symbol = "\\ "
                else:
                    start_symbol = "| "
            
            if (j == cols - 1):
                if i == 0:
                    end_symbol = "  \\"
                elif i == rows - 1:
                    end_symbol = "  /"
                else:
                    end_symbol = "  |"

            print(start_symbol + "{:6.8f}".format(M[i, j]), end="")
            print(end_symbol + ("\n" if j == cols - 1 else "  "), end="")

    print("\n")

printMatrix(matrix)

Matrix[12][12]
/ 1.00000000  0.33333333  0.25000000  0.20000000  0.16666667  0.14285714  0.12500000  0.11111111  0.10000000  0.09090909  0.08333333  0.07692308  \
| 0.20000000  1.00000000  0.14285714  0.12500000  0.11111111  0.10000000  0.09090909  0.08333333  0.07692308  0.07142857  0.06666667  0.06250000  |
| 0.10000000  0.09090909  1.00000000  0.07692308  0.07142857  0.06666667  0.06250000  0.05882353  0.05555556  0.05263158  0.05000000  0.04761905  |
| 0.05882353  0.05555556  0.05263158  1.00000000  0.04761905  0.04545455  0.04347826  0.04166667  0.04000000  0.03846154  0.03703704  0.03571429  |
| 0.03846154  0.03703704  0.03571429  0.03448276  1.00000000  0.03225806  0.03125000  0.03030303  0.02941176  0.02857143  0.02777778  0.02702703  |
| 0.02702703  0.02631579  0.02564103  0.02500000  0.02439024  1.00000000  0.02325581  0.02272727  0.02222222  0.02173913  0.02127660  0.02083333  |
| 0.02000000  0.01960784  0.01923077  0.01886792  0.01851852  0.01818182  1.00000000  0.01754386 

In [138]:
# Checking if LU method is possible
LU = True

for i in range(1, n + 1):
    minor = ((matrix[0:(i)].T)[0:(i)]).T    # expression for minor
    if np.linalg.det(minor) == 0.0:         # if at least one minor is zero then LU isn't possible
        LU = False
        break

In [139]:
# LU method

if LU:  
  L = np.zeros ((n, n))
  U = np.zeros ((n ,n))

  for i in range(n):
      L[i][i] = 1

  for i in range (n):
      for j in range (n):
          if i <= j:
              U[i][j] = matrix[i][j] - np.sum ([L[i][k] * U[k][j] for k in range(i)])

          else:
              L[i][j] = (matrix[i][j] - np.sum ([L[i][k] * U[k][j] for k in range(j)])) / U[j][j]

  y = np.zeros (n)

  for i in range (n):
      y[i] = (f[i] - np.sum ([y[k] * L[i][k] for k in range (i)]))

  x[n - 1] = y[n - 1] / U[n - 1][n - 1]
  for i in range (n - 2, -1, -1):
      x[i] = (y[i] - np.sum ([x[k] * U[i][k] for k in range (i + 1, n)])) / U[i][i]

  print ("Roots of LU method:")
  print (x)
  print ("Solution Discrepancy |A * x - f|:", np.linalg.norm (np.matmul (matrix, x) - f))

else: print("LU is not possible")

Roots of LU method:
[0.75537042 0.23879963 0.17890325 0.15086988 0.13170938 0.1170332
 0.10527117 0.09560544 0.08752166 0.08066593 0.074783   0.06968322]
Solution Discrepancy |A * x - f|: 6.655553224256149e-17


In [140]:
# Relaxation method

w = random.uniform(1, 2)
err = 1e-15                           
N = 0                                 # number of iterations
L = np.zeros ((n, n))
D = np.zeros ((n, n))
U = np.zeros ((n, n))

for i in range (n):
    for j in range (n):
        if i < j:
            U[i][j] = matrix[i][j]
        if i == j:
            D[i][j] = matrix[i][j]
        if i > j:
            L[i][j] = matrix[i][j]

x_prev = np.random.random (n)          #initial approximation
x = np.random.random (n)

while np.linalg.norm (x - x_prev) > err:      # stop criteria
    x_prev = x
    x = - np.matmul (np.matmul (np.linalg.inv (D + w * L), (w - 1) * D + w * U), x_prev) + w * np.matmul (np.linalg.inv (D + w * L), f)
    N += 1

print ("Roots of relaxation method")
print (x)
print ("Solution Discrepancy |A * x - f|:", np.linalg.norm (np.matmul (matrix, x) - f))
print ("Number of iterations:", N)
print(w)

Roots of relaxation method
[0.75537042 0.23879963 0.17890325 0.15086988 0.13170938 0.1170332
 0.10527117 0.09560544 0.08752166 0.08066593 0.074783   0.06968322]
Solution Discrepancy |A * x - f|: 3.626858354025536e-16
Number of iterations: 41
1.339791812393254


In [132]:
# λ_min and λ_max
eig, _ = np.linalg.eig(matrix)
print("min|λ(A)|, max|λ(A)| = ", eig.min(), eig.max())

# μ = ||A||*||A^(-1)|| in third form
eig, _ = np.linalg.eig(np.matmul(matrix.T, matrix))
norm_m = np.sqrt(eig.max())
print("μ = ||A||*||A^(-1)|| = ", norm_m)

min|λ(A)|, max|λ(A)| =  0.7357016342479968 1.519941391528549
μ = ||A||*||A^(-1)|| =  1.617256887091802
